In [ ]:
#| hide

# Test creating a tenant database
from fh_saas.db_tenant import get_or_create_tenant_db, init_tenant_core_schema, TenantUser
from fh_saas.db_host import timestamp, gen_id
from dotenv import load_dotenv
load_dotenv()

# Create/connect to test tenant
tenant_db = get_or_create_tenant_db("test_tenant_001", "Test Company")

# Initialize core schema
core_tables = init_tenant_core_schema(tenant_db)

print(f"\n✅ Core tenant tables created:")
print(f"   - core_tenant_users: {len(core_tables['tenant_users']())} users")
print(f"   - core_permissions: {len(core_tables['permissions']())} permissions")
print(f"   - core_settings: {len(core_tables['settings']())} settings")

# Test inserting a tenant user
test_user = TenantUser(
    id=gen_id(),
    display_name="Test User",
    local_role="admin",
    created_at=timestamp()
)
try:
    core_tables['tenant_users'].insert(test_user)
    print(f"\n✅ Test user created: {test_user.display_name}")
except Exception as e:
    tenant_db.conn.rollback()
    print(f"\n✅ Test user already exists: {test_user.display_name}")

ℹ️  Tenant exists: Test Company

✅ Core tenant tables created:
   - core_tenant_users: 35 users
   - core_permissions: 0 permissions
   - core_settings: 0 settings

✅ Test user created: Test User


In [ ]:
#| hide

# Test webhook tables
from fh_saas.db_tenant import WebhookEvent, WebhookSecret
import sqlalchemy as sa
from sqlalchemy.exc import IntegrityError

print("\n" + "="*60)
print("Testing Webhook Tables")
print("="*60)

# Reconnect to get fresh transaction
tenant_db = get_or_create_tenant_db("test_tenant_001", "Test Company")
core_tables = init_tenant_core_schema(tenant_db)

# Force rollback any failed transaction
try:
    tenant_db.conn.rollback()
except:
    pass

# Clean up any existing test data first
try:
    tenant_db.conn.execute(
        core_tables['webhook_secrets'].table.delete().where(
            core_tables['webhook_secrets'].table.c.source == 'stripe'
        )
    )
    tenant_db.conn.execute(
        core_tables['webhook_events'].table.delete().where(
            core_tables['webhook_events'].table.c.idempotency_key == 'stripe_evt_test_001'
        )
    )
    tenant_db.conn.commit()
    print("🧹 Cleaned up existing test data")
except Exception as e:
    tenant_db.conn.rollback()
    print(f"⚠️  Cleanup note: {str(e)[:50]}")

# Test webhook event insertion
test_webhook = WebhookEvent(
    webhook_id=gen_id(),
    source='stripe',
    event_type='payment.succeeded',
    payload_json='{"amount": 5000, "currency": "usd"}',
    signature='abc123def456',
    idempotency_key='stripe_evt_test_001',
    status='pending',
    created_at=timestamp()
)
core_tables['webhook_events'].insert(test_webhook)
tenant_db.conn.commit()  # Commit successful insert
print(f"✅ Webhook event created: {test_webhook.event_type}")

# Test idempotency constraint
try:
    duplicate_webhook = WebhookEvent(
        webhook_id=gen_id(),
        source='stripe',
        event_type='payment.succeeded',
        payload_json='{"amount": 5000}',
        signature='different_sig',
        idempotency_key='stripe_evt_test_001',  # Same key!
        status='pending',
        created_at=timestamp()
    )
    core_tables['webhook_events'].insert(duplicate_webhook)
    tenant_db.conn.commit()
    print("❌ ERROR: Duplicate idempotency key should have been rejected")
except IntegrityError:
    tenant_db.conn.rollback()  # Rollback the failed transaction
    print("✅ Idempotency constraint works - duplicate key rejected")
except Exception as e:
    tenant_db.conn.rollback()
    print(f"⚠️  Unexpected error: {e}")

# Test webhook secret insertion
test_secret = WebhookSecret(
    source='stripe',
    secret_key='whsec_test_secret_key',
    is_active=True,
    created_at=timestamp()
)
core_tables['webhook_secrets'].insert(test_secret)
tenant_db.conn.commit()  # Commit successful insert
print(f"✅ Webhook secret stored for source: {test_secret.source}")

# Verify tables exist and have data - rollback first to clear any bad state
try:
    tenant_db.conn.rollback()
except:
    pass

webhook_count = len(core_tables['webhook_events']())
secret_count = len(core_tables['webhook_secrets']())
print(f"\n📊 Final counts:")
print(f"   - webhook_events: {webhook_count} events")
print(f"   - webhook_secrets: {secret_count} secrets")

print("\n✅ All webhook table tests passed!")
print("="*60)


Testing Webhook Tables
ℹ️  Tenant exists: Test Company
🧹 Cleaned up existing test data
✅ Webhook event created: payment.succeeded
❌ ERROR: Duplicate idempotency key should have been rejected
✅ Webhook secret stored for source: stripe

📊 Final counts:
   - webhook_events: 2 events
   - webhook_secrets: 1 secrets

✅ All webhook table tests passed!
